<a href="https://colab.research.google.com/github/rosafilgueira/PyCodeSearch/blob/main/Registry_search_multimodel_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Transformer models for code


## UnixCoders and reacc Models

We are going to use two models:
-  Lazyhope/unixcoder-nine-advtest, wich is used by the original RepoSim pipeline - **text-to-code search**

- microsoft/reacc-py-retriever -- **code-completion**



In [3]:
%pip install tensorflow
%pip install -U accelerate
%pip install docarray
%pip install pandas
%pip install torch
%pip install transformers
%pip install tqdm
%pip install scikit-learn
%pip install sentencepiece


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note

## Add database example

In [4]:
import pandas as pd

# Define the codes and their corresponding docstrings
codes = [
    "def add(a, b):\n    return a + b",
    "def subtract(a, b):\n    return a - b",
    "def multiply(a, b):\n    return a * b",
    "def divide(a, b):\n    return a / b",
    "def power(a, b):\n    return a ** b",
    "def modulus(a, b):\n    return a % b"
    
]

docs = [
    "This function adds two numbers.",
    "This function subtracts the second number from the first.",
    "This function multiplies two numbers.",
    "This function divides the first number by the second.",
    "This function raises the first number to the power of the second.",
    "This function returns the remainder when the first number is divided by the second."
]

# Create the dataframe
registry = pd.DataFrame({
    'code': codes,
    'doc': docs
})

### dataframe
registry

,code,doc
0,"def add(a, b):\n return a + b",This function adds two numbers.
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...


## Load two models for two different tasks

- reacc-py-retriever -- for code-completion search
- model_text_to_code -- for text-to-code search

In [5]:
#NEW: I have included two new imports here
from transformers import pipeline, AutoModel, AutoTokenizer

## Note: this pipeline uses "Lazyhope/unixcoder-nine-advtest" underneed
model_text_to_code = pipeline(
    model="Lazyhope/RepoSim",
    trust_remote_code=True,
    device_map="auto")

#NEW
model_name= "microsoft/reacc-py-retriever"
model_code_completion = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/cs/home/dr207/Documents/CS4000/Laminar/dispel4py-client/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-20 10:40:43.627807: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 10:40:44.745643: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dr207/usr/lib:/cs/home/dr207/usr/lib
2024-03-20 10:40:44.745660: I tensorflow/compiler/xla/stream_e

[*] Consider setting GitHub token to avoid hitting rate limits. 
For more info, see: https://docs.github.com/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token


config.json: 100%|██████████| 689/689 [00:00<00:00, 1.35MB/s]
pytorch_model.bin: 100%|██████████| 496M/496M [00:12<00:00, 39.9MB/s] 
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/reacc-py-retriever and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
tokenizer_config.json: 100%|██████████| 1.40k/1.40k [00:00<00:00, 2.21MB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 4.07MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 2.06MB/s]
added_tokens.json: 100%|██████████| 60.0/60.0 [00:00<00:00, 96.2kB/s]
special_tokens_map.json: 100%|██████████| 842/842 [00:00<00:00, 1.23MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or tra

## Turn code and docstring into torch embeddings

In [6]:
#NEW

import torch

def encode(string, model_type):
    if model_type == 1:
        with torch.no_grad():
            embedding = model_text_to_code.encode(string, 512)
        final_t=embedding.squeeze()

    else:
        with torch.no_grad():
            inputs = tokenizer(string, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
            outputs = model_code_completion(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1)  # You can use any pooling strategy here
        final_t = embedding.squeeze()
    return final_t

## model_type =1 -- for text-to_code
registry["doc_embeddings"] = registry["doc"].apply(encode, model_type=1)
## model_type=2 -- for code-completion
registry["code_embeddings"] = registry["code"].apply(encode, model_type=1)
registry

,code,doc,doc_embeddings,code_embeddings
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[tensor(-1.5903), tensor(-0.3973), tensor(3.99...","[tensor(-0.4317), tensor(-0.0293), tensor(2.66..."
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[tensor(-2.4678), tensor(-2.2801), tensor(0.66...","[tensor(-1.2667), tensor(-0.6932), tensor(-0.4..."
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[tensor(-2.3068), tensor(1.3498), tensor(3.078...","[tensor(-1.0338), tensor(1.4185), tensor(1.093..."
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[tensor(-3.0391), tensor(-0.0748), tensor(1.79...","[tensor(-0.9333), tensor(2.0973), tensor(0.789..."
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[tensor(-2.9461), tensor(-0.5175), tensor(2.37...","[tensor(-2.0816), tensor(1.8068), tensor(0.650..."
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[tensor(-2.4903), tensor(-1.3584), tensor(1.60...","[tensor(-0.2191), tensor(0.7266), tensor(-0.20..."


### Text-to-code Similarity

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [8]:
# Compute user query embeddings
user_query_docs = "Function that will find the sum of two numbers"
#NEW -- indicate the model_type=1 for unixcoder (text-to-code)
user_query_docs_emb = encode(user_query_docs, model_type=2)


# Convert document embeddings to numpy arrays
registry["doc_embeddings"] = registry["doc_embeddings"].apply(lambda x: np.array(x))

# Compute cosine similarity
user_query_emb = np.array(user_query_docs_emb)
cos_similarities = cosine_similarity(user_query_emb.reshape(1, -1), np.vstack(registry["doc_embeddings"]))

# Add cosine similarity scores as a new column
registry_doc=registry.copy()
registry_doc["cosine_similarity_doc"] = cos_similarities[0]

# Sort the dataframe based on cosine similarity
sorted_df = registry_doc.sort_values(by="cosine_similarity_doc", ascending=False)

# Retrieve the top 5 most similar documents
top_5_similar_docs = sorted_df.head(5)
top_5_similar_docs

,code,doc,doc_embeddings,code_embeddings,cosine_similarity_doc
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-1.590325, -0.39731324, 3.997744, 2.6590736, ...","[tensor(-0.4317), tensor(-0.0293), tensor(2.66...",-0.012405
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-3.0390859, -0.07484988, 1.7925602, 0.2812554...","[tensor(-0.9333), tensor(2.0973), tensor(0.789...",-0.012467
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[-2.490343, -1.3584495, 1.6064605, -0.03004119...","[tensor(-0.2191), tensor(0.7266), tensor(-0.20...",-0.025223
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-2.4677699, -2.280132, 0.6681174, 1.4311584, ...","[tensor(-1.2667), tensor(-0.6932), tensor(-0.4...",-0.032303
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[-2.946074, -0.51749504, 2.3797052, 2.3707116,...","[tensor(-2.0816), tensor(1.8068), tensor(0.650...",-0.035086


In [9]:
top_5_similar_docs

,code,doc,doc_embeddings,code_embeddings,cosine_similarity_doc
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-1.590325, -0.39731324, 3.997744, 2.6590736, ...","[tensor(-0.4317), tensor(-0.0293), tensor(2.66...",-0.012405
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-3.0390859, -0.07484988, 1.7925602, 0.2812554...","[tensor(-0.9333), tensor(2.0973), tensor(0.789...",-0.012467
5,"def modulus(a, b):\n return a % b",This function returns the remainder when the f...,"[-2.490343, -1.3584495, 1.6064605, -0.03004119...","[tensor(-0.2191), tensor(0.7266), tensor(-0.20...",-0.025223
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-2.4677699, -2.280132, 0.6681174, 1.4311584, ...","[tensor(-1.2667), tensor(-0.6932), tensor(-0.4...",-0.032303
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[-2.946074, -0.51749504, 2.3797052, 2.3707116,...","[tensor(-2.0816), tensor(1.8068), tensor(0.650...",-0.035086


### Code Completion

In [10]:
# Compute user query embeddings
user_query_code = "def add_numbers(a, b):\n return a "
#NEW -- indicate the model_type=2 for ReACC (code-completion)
user_query_code_emb = encode(user_query_code, model_type=2)
# Convert document embeddings to numpy arrays
registry["code_embeddings"] = registry["code_embeddings"].apply(lambda x: np.array(x))

# Compute cosine similarity
user_query_emb_c = np.array(user_query_code_emb)
cos_similarities = cosine_similarity(user_query_emb_c.reshape(1, -1), np.vstack(registry["code_embeddings"]))

# Add cosine similarity scores as a new column
registry_code=registry.copy()
registry_code["cosine_similarity_code_2"] = cos_similarities[0]

# Sort the dataframe based on cosine similarity
sorted_df_code = registry_code.sort_values(by="cosine_similarity_code_2", ascending=False)

# Retrieve the top 5 most similar documents
top_5_similar_code = sorted_df_code.head(5)
top_5_similar_code

,code,doc,doc_embeddings,code_embeddings,cosine_similarity_code_2
0,"def add(a, b):\n return a + b",This function adds two numbers.,"[-1.590325, -0.39731324, 3.997744, 2.6590736, ...","[-0.4316707, -0.029330524, 2.6694558, 3.79384,...",0.034617
3,"def divide(a, b):\n return a / b",This function divides the first number by the ...,"[-3.0390859, -0.07484988, 1.7925602, 0.2812554...","[-0.93327546, 2.0973008, 0.7891953, 1.654753, ...",0.028124
4,"def power(a, b):\n return a ** b",This function raises the first number to the p...,"[-2.946074, -0.51749504, 2.3797052, 2.3707116,...","[-2.0816395, 1.8067635, 0.6504532, 2.820747, 1...",0.011637
1,"def subtract(a, b):\n return a - b",This function subtracts the second number from...,"[-2.4677699, -2.280132, 0.6681174, 1.4311584, ...","[-1.266672, -0.6931733, -0.43216208, 2.7053928...",0.004685
2,"def multiply(a, b):\n return a * b",This function multiplies two numbers.,"[-2.3068378, 1.3498034, 3.0785217, 1.3164998, ...","[-1.0338337, 1.418475, 1.0938523, 3.0212605, 0...",-0.013502


# Code Sumarization

In [11]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')

# from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

# pipeline = SummarizationPipeline(
#     model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask_finetune"),
#     tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask_finetune", skip_special_tokens=True),
#     device=0
# )


tokenizer_config.json: 100%|██████████| 1.48k/1.48k [00:00<00:00, 1.53MB/s]
vocab.json: 100%|██████████| 703k/703k [00:00<00:00, 41.0MB/s]
merges.txt: 100%|██████████| 294k/294k [00:00<00:00, 46.6MB/s]
added_tokens.json: 100%|██████████| 2.00/2.00 [00:00<00:00, 3.79kB/s]
special_tokens_map.json: 100%|██████████| 12.5k/12.5k [00:00<00:00, 20.1MB/s]
config.json: 100%|██████████| 902/902 [00:00<00:00, 1.04MB/s]
pytorch_model.bin: 100%|██████████| 892M/892M [00:07<00:00, 112MB/s]  


In [12]:
def generate_summary(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    generated_ids = model.generate(input_ids, max_length=20)
    summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return summary

In [13]:
registry_summary=registry.copy()
# Assuming the registry dataframe is already defined
registry_summary["summarization"] = registry["code"].apply(generate_summary)
# registry_summary


Improvements to code summarisation

In [14]:
# Define the codes and their corresponding docstrings
PE_processes = [

    '''def process(self, inputs):
        self.counter += 1
        for i in range(self.numIterations):
            self.write("output", f"{self.counter}:{i}")''',
    
    '''def process(self, inputs):
        result = ""
        for inp in self.inputconnections:
            if inp in inputs:
                result += str(inputs[inp])
        if result:
            return {"output": result}''',

    '''def process(self, inputs=None):
        word = random.choice(RandomWordProducer.words)
        outputs = {"output": [word]}
        return outputs''',

    '''def _process(self, inputs):
        result= random.randint(1, 1000)
        return result''' ,

    '''def process(self, inputs):
        try:
            metadata = inputs["metadata"]
            self.log(f"Logging metadata: {str(metadata)[:300]}")
        except:
            self.log(traceback.format_exc())''',
    '''def _process(self, inputs):
        for i in range(self.num_output):
            self.write("output", i)''',
    
    
    
    
    
    '''def _process(self, data):
        article = data
        avg_score = 0
        count = 0
        article_word = nltk.word_tokenize(article["text"])
        sent_score = 0  # sentiment score della frase
        for word in article_word:
            word = word.rstrip('?:!.,;"!@')
            word = word.replace("\n", "")
            if not (word.encode("utf-8", "ignore") == ""):
                if word.encode("utf-8") in self.sentiment.keys():
                    sent_score = sent_score + float(self.sentiment[word])
                    count += 1
        if count != 0:
            avg_score = sent_score / count
        else:
            avg_score = sent_score

        # print("AFINNSentimeScore %s %s %s" % (article, str(avg_score),self.method))
        return (article, avg_score, self.method)
    ''',

    '''def _process(self, data):

        # This PE calculates the average sentiment score based on the word sentiment file provided.
        
        article = data
        avg_score = 0
        count = 0
        article_word = nltk.word_tokenize(article["text"])
        sent_score = 0  # sentiment score della frase

        # for every word in the article we iterate through and asign it a sentiment score if we recognise it
        for word in article_word:
            # cleaning the word
            word = word.rstrip('?:!.,;"!@')
            word = word.replace("\n", "")
            # ignores words that are not in the appropriate encoding
            if not (word.encode("utf-8", "ignore") == ""):
                # if we have data on the word we retrieve it's sentiment score
                if word.encode("utf-8") in self.sentiment.keys():
                    sent_score = sent_score + float(self.sentiment[word])
                    count += 1
        # check that we have recognised any words, then find a median of the score
        if count != 0:
            avg_score = sent_score / count
        # no words recognised, so the score is 0
        else:
            avg_score = sent_score

        # print("AFINNSentimeScore %s %s %s" % (article, str(avg_score),self.method))
        return (article, avg_score, self.method)''',



    
    '''def _process(self, data):
        article = data
        avg_score = 0
        count = 0
        article_word = nltk.word_tokenize(article["text"])
        sent_score = 0  # sentiment score della frase

        # for every word in the article we iterate through and asign it a sentiment score if we recognise it
        for word in article_word:
            # cleaning the word
            word = word.rstrip('?:!.,;"!@')
            word = word.replace("\n", "")
            # ignores words that are not in the appropriate encoding
            if not (word.encode("utf-8", "ignore") == ""):
                # if we have data on the word we retrieve it's sentiment score
                if word.encode("utf-8") in self.sentiment.keys():
                    sent_score = sent_score + float(self.sentiment[word])
                    count += 1
        # check that we have recognised any words, then find a median of the score
        if count != 0:
            avg_score = sent_score / count
        # no words recognised, so the score is 0
        else:
            avg_score = sent_score

        # print("AFINNSentimeScore %s %s %s" % (article, str(avg_score),self.method))
        return (article, avg_score, self.method)
''',
    '''def process(self, inputs):
        twitterData = inputs["input"]
        article_file = open(path.join(ROOT_DIR, twitterData))

        for line in article_file:
            fields = line.split("\t")
            location = fields[3]
            text = fields[4]
            self.count += 1
            return_article = {"text": text, "location": location}
            self.write("output", return_article)

        print(f"Total articles found {self.count}")'''


    
]

PE_full= [
    '''class NumberProducer(GenericPE):
    def __init__(self, numIterations=1):
        GenericPE.__init__(self)
        self.outputconnections = {"output": {NAME: "output", TYPE: ["number"]}}
        self.counter = 0
        self.numIterations = numIterations

    def process(self, inputs):
        self.counter += 1
        for i in range(self.numIterations):
            self.write("output", f"{self.counter}:{i}")''',
    
    '''class TestTwoInOneOut(GenericPE):
    

    def __init__(self):
        GenericPE.__init__(self)
        self.inputconnections = {"input0": {NAME: "input0"}, "input1": {NAME: "input1"}}
        self.outputconnections = {"output": {NAME: "output", TYPE: ["result"]}}

    def process(self, inputs):
        result = ""
        for inp in self.inputconnections:
            if inp in inputs:
                result += str(inputs[inp])
        if result:
            return {"output": result}''',

    
    '''class RandomWordProducer(GenericPE):
    
    words = [
        "Dispel4Py",
        "Earthquake",
        "Computing",
        "Seismology",
        "Modelling",
        "Analysis",
        "Infrastructure",
    ]

    def __init__(self):
        GenericPE.__init__(self)
        out1 = {}
        out1[NAME] = "output"
        out1[TYPE] = ["word"]
        self.outputconnections["output"] = out1

    def process(self, inputs=None):
        word = random.choice(RandomWordProducer.words)
        outputs = {"output": [word]}
        return outputs''',


    '''class NumberProducer(ProducerPE):
    def __init__(self):
        ProducerPE.__init__(self)
        
    def _process(self, inputs):
        # this PE produces one input
        result= random.randint(1, 1000)
        return result''',

    '''
class ProvenanceLogger(GenericPE):
    def __init__(self):
        GenericPE.__init__(self)
        self.inputconnections = {"metadata": {NAME: "metadata"}}

    def process(self, inputs):
        try:
            metadata = inputs["metadata"]
            self.log(f"Logging metadata: {str(metadata)[:300]}")
        except:
            self.log(traceback.format_exc())
''',

'''class TestMultiProducer(GenericPE):
    def __init__(self, num_output=10):
        GenericPE.__init__(self)
        self._add_output("output")
        self.num_output = num_output

    def _process(self, inputs):
        for i in range(self.num_output):
            self.write("output", i)
''',        
    
    
    
    
    
        '''class AFINNSentimeScore(IterativePE):
    def __init__(self, sentimentData):
        IterativePE.__init__(self)
        
        afinnfile = open(path.join(ROOT_DIR, sentimentData))
        self.sentiment = {}
        for line in afinnfile:
            term, score = line.split(
                "\t"
            )  # The file is tab-delimited. "\t" means "tab character"
            self.sentiment[term] = float(score)  # Convert the score to an integer.
        self.method = "AFINN"

    def _process(self, data):
        article = data
        avg_score = 0
        count = 0
        article_word = nltk.word_tokenize(article["text"])
        sent_score = 0  # sentiment score della frase
        for word in article_word:
            word = word.rstrip('?:!.,;"!@')
            word = word.replace("\n", "")
            if not (word.encode("utf-8", "ignore") == ""):
                if word.encode("utf-8") in self.sentiment.keys():
                    sent_score = sent_score + float(self.sentiment[word])
                    count += 1
        if count != 0:
            avg_score = sent_score / count
        else:
            avg_score = sent_score

        # print("AFINNSentimeScore %s %s %s" % (article, str(avg_score),self.method))
        return (article, avg_score, self.method)''',


'''class AFINNSentimeScore(IterativePE):
    # This PE takes an article and calculates it's average sentiment score based on the word sentiment file provided
    def __init__(self, sentimentData):
        IterativePE.__init__(self)
        
        afinnfile = open(path.join(ROOT_DIR, sentimentData))
        self.sentiment = {}
        for line in afinnfile:
            term, score = line.split(
                "\t"
            )  # The file is tab-delimited. "\t" means "tab character"
            self.sentiment[term] = float(score)  # Convert the score to an integer.
        self.method = "AFINN"

    def _process(self, data):
        article = data
        avg_score = 0
        count = 0
        article_word = nltk.word_tokenize(article["text"])
        sent_score = 0  # sentiment score della frase
        for word in article_word:
            word = word.rstrip('?:!.,;"!@')
            word = word.replace("\n", "")
            if not (word.encode("utf-8", "ignore") == ""):
                if word.encode("utf-8") in self.sentiment.keys():
                    sent_score = sent_score + float(self.sentiment[word])
                    count += 1
        if count != 0:
            avg_score = sent_score / count
        else:
            avg_score = sent_score

        # print("AFINNSentimeScore %s %s %s" % (article, str(avg_score),self.method))
        return (article, avg_score, self.method)''',

        '''class AFINNSentimeScore(IterativePE):
    def __init__(self, sentimentData):
        IterativePE.__init__(self)
        
        afinnfile = open(path.join(ROOT_DIR, sentimentData))
        self.sentiment = {}
        for line in afinnfile:
            term, score = line.split(
                "\t"
            )  # The file is tab-delimited. "\t" means "tab character"
            self.sentiment[term] = float(score)  # Convert the score to an integer.
        self.method = "AFINN"

    def _process(self, data):
        article = data
        avg_score = 0
        count = 0
        article_word = nltk.word_tokenize(article["text"])
        sent_score = 0  # sentiment score della frase

        # for every word in the article we iterate through and asign it a sentiment score if we recognise it
        for word in article_word:
            # cleaning the word
            word = word.rstrip('?:!.,;"!@')
            word = word.replace("\n", "")
            # ignores words that are not in the appropriate encoding
            if not (word.encode("utf-8", "ignore") == ""):
                # if we have data on the word we retrieve it's sentiment score
                if word.encode("utf-8") in self.sentiment.keys():
                    sent_score = sent_score + float(self.sentiment[word])
                    count += 1
        # check that we have recognised any words, then find a median of the score
        if count != 0:
            avg_score = sent_score / count
        # no words recognised, so the score is 0
        else:
            avg_score = sent_score

        # print("AFINNSentimeScore %s %s %s" % (article, str(avg_score),self.method))
        return (article, avg_score, self.method)''',


        '''class ReadData(GenericPE):
    def __init__(self):
        GenericPE.__init__(self)
        self._add_output("output")
        self.count = 0

    def process(self, inputs):
        twitterData = inputs["input"]
        article_file = open(path.join(ROOT_DIR, twitterData))

        for line in article_file:
            fields = line.split("\t")
            location = fields[3]
            text = fields[4]
            self.count += 1
            return_article = {"text": text, "location": location}
            self.write("output", return_article)

        print(f"Total articles found {self.count}")
'''



]


PE_names  = ["NumberProducer", "TestTwoInOneOut", "RandomWordProducer", "RandomNumberProducer", "ProvenanceLogger",
                 "TestMultiProducer", "AFINNSentimeScore", "AFINNSentimeScoreCommented", "AFinnSentimeScoreCommented2",
                    "ReadData"]

In [15]:
chatGPT3Summaries = ["The NumberProducer PE, a subclass of GenericPE, generates and outputs a sequence of numbers based on the specified number of iterations.",
"The TestTwoInOneOut PE, extending GenericPE, takes two input connections, combines their values into a single result, and provides it as an output.",
"The RandomWordProducer PE, based on the GenericPE class, generates and provides random words from a predefined list as output.",
"The NumberProducer PE, a subclass of ProducerPE, generates and provides a random number between 1 and 1000 as its output.",
"The ProvenanceLogger PE, extending the GenericPE class, is designed to log metadata and handle exceptions during its processing.",
"The TestMultiProducer PE, extending GenericPE, generates multiple output values based on the specified number of outputs.",
"The AFINNSentimeScore PE, inherited from IterativePE, calculates the sentiment score of text data using the AFINN lexicon and returns the average sentiment score for each article.",
"The AFINNSentimeScore PE, inherited from IterativePE, computes the average sentiment score of an article by analyzing the sentiment of its text using a provided sentiment lexicon.",
"The AFINNSentimeScore PE, derived from IterativePE, computes sentiment scores for text data using the AFINN lexicon and includes a method to calculate the average sentiment score for each article.",
"The ReadData PE, extending GenericPE, reads and processes data from a file, extracting text and location information from each line and providing it as output, while keeping track of the number of processed ",]

In [16]:
# Create the dataframe
registryLam = pd.DataFrame({
    'code': PE_full,
    'code_embedding': "",
    'code_line_embeddings' : "",
    'code_line_splits' : "",
    'process' : PE_processes,
    'class': PE_names,
    'summary' : "" ,
    'mod_summary'  : "",
    'sum_doc' : "",
    'mod_sum_doc' : "",
    'full_summary' : "",
    'mod_full_sum' : "",
    'gpt_sum' : chatGPT3Summaries,
    'gpt_sum_doc' : "" 

})
# provides more specificty as we know that they are PEs and not just generic classes
def improve_summary(dfRow, sumCol):
    return dfRow[sumCol].replace(" class ", " PE ")


registryLam['code_embedding'] = registryLam['code'].apply(encode, model_type=2)

registryLam['full_summary'] = registryLam['code'].apply(generate_summary)
registryLam['mod_full_sum'] = registryLam.apply(improve_summary, sumCol='full_summary', axis=1)
registryLam['full_sum_doc'] = registryLam['full_summary'].apply(encode, model_type=1).apply(lambda x: np.array(x))
registryLam['mod_full_sum_doc'] = registryLam['mod_full_sum'].apply(encode, model_type=1).apply(lambda x: np.array(x))



registryLam['summary'] = registryLam['process'].apply(generate_summary)
registryLam['mod_summary'] = registryLam.apply(improve_summary, sumCol="summary", axis=1)
registryLam['sum_doc'] = registryLam['summary'].apply(encode, model_type=1).apply(lambda x: np.array(x))
registryLam['mod_sum_doc'] = registryLam['mod_summary'].apply(encode, model_type=1).apply(lambda x: np.array(x))

registryLam['gpt_sum_doc'] = registryLam['gpt_sum'].apply(encode, model_type=1).apply(lambda x: np.array(x))
# registryLam

In [17]:
# Compute user query embeddings


def columnSim(inputString, columnName, modelType):

    #NEW -- indicate the model_type=1 for unixcoder (text-to-code)
    user_query_docs_emb = encode(inputString, model_type=modelType)

    # Compute cosine similarity
    user_query_emb = np.array(user_query_docs_emb)
    cos_similarities = cosine_similarity(user_query_emb.reshape(1, -1), np.vstack(registryLam[columnName]))

    # Add cosine similarity scores as a new column
    registry_doc=registryLam.copy().drop(['code_embedding', 'code_line_embeddings', 'process', 'sum_doc', 'mod_sum_doc','full_sum_doc', 'mod_full_sum_doc', 'gpt_sum_doc'], axis=1)
    # registry_doc.drop('code', axis=1)
    registry_doc["cosine_similarity_doc"] = cos_similarities[0]

    # Sort the dataframe based on cosine similarity
    sorted_df = registry_doc.sort_values(by="cosine_similarity_doc", ascending=False)

    # Retrieve the top 5 most similar documents
    top_5_similar_docs = sorted_df.head(5)
    return top_5_similar_docs








In [18]:
columnSim('log the metadata from incomming connections','sum_doc', 1)

,code,code_line_splits,class,summary,mod_summary,full_summary,mod_full_sum,gpt_sum,cosine_similarity_doc
4,\nclass ProvenanceLogger(GenericPE):\n def ...,,ProvenanceLogger,Log the nagios.,Log the nagios.,A logger for the missing sequence sequence.,A logger for the missing sequence sequence.,"The ProvenanceLogger PE, extending the Generic...",0.423034
9,class ReadData(GenericPE):\n def __init__(s...,,ReadData,Process the nagios. txt file and return a dict...,Process the nagios. txt file and return a dict...,This class reads the data from the twitter and...,This PE reads the data from the twitter and wr...,"The ReadData PE, extending GenericPE, reads an...",0.111964
6,class AFINNSentimeScore(IterativePE):\n def...,,AFINNSentimeScore,Process the nltk tag and return the average sc...,Process the nltk tag and return the average sc...,AFASTA AFINNSentimeScore - AFASTA AFINNS,AFASTA AFINNSentimeScore - AFASTA AFINNS,"The AFINNSentimeScore PE, inherited from Itera...",0.075626
8,class AFINNSentimeScore(IterativePE):\n def...,,AFinnSentimeScoreCommented2,Process the nltk tag and return the article th...,Process the nltk tag and return the article th...,AFASTIME - SENTIME - SENTIME - SENTIME - S,AFASTIME - SENTIME - SENTIME - SENTIME - S,"The AFINNSentimeScore PE, derived from Iterati...",0.074254
1,class TestTwoInOneOut(GenericPE):\n \n\n ...,,TestTwoInOneOut,Process the sequence of sequence of sequence o...,Process the sequence of sequence of sequence o...,This class is a base class for test 2 - in - o...,This PE is a base PE for test 2 - in - one - o...,"The TestTwoInOneOut PE, extending GenericPE, t...",0.054102


In [19]:
columnSim('log the metadata from incomming connections', 'mod_sum_doc', 1)


,code,code_line_splits,class,summary,mod_summary,full_summary,mod_full_sum,gpt_sum,cosine_similarity_doc
4,\nclass ProvenanceLogger(GenericPE):\n def ...,,ProvenanceLogger,Log the nagios.,Log the nagios.,A logger for the missing sequence sequence.,A logger for the missing sequence sequence.,"The ProvenanceLogger PE, extending the Generic...",0.423034
9,class ReadData(GenericPE):\n def __init__(s...,,ReadData,Process the nagios. txt file and return a dict...,Process the nagios. txt file and return a dict...,This class reads the data from the twitter and...,This PE reads the data from the twitter and wr...,"The ReadData PE, extending GenericPE, reads an...",0.111964
6,class AFINNSentimeScore(IterativePE):\n def...,,AFINNSentimeScore,Process the nltk tag and return the average sc...,Process the nltk tag and return the average sc...,AFASTA AFINNSentimeScore - AFASTA AFINNS,AFASTA AFINNSentimeScore - AFASTA AFINNS,"The AFINNSentimeScore PE, inherited from Itera...",0.075626
8,class AFINNSentimeScore(IterativePE):\n def...,,AFinnSentimeScoreCommented2,Process the nltk tag and return the article th...,Process the nltk tag and return the article th...,AFASTIME - SENTIME - SENTIME - SENTIME - S,AFASTIME - SENTIME - SENTIME - SENTIME - S,"The AFINNSentimeScore PE, derived from Iterati...",0.074254
1,class TestTwoInOneOut(GenericPE):\n \n\n ...,,TestTwoInOneOut,Process the sequence of sequence of sequence o...,Process the sequence of sequence of sequence o...,This class is a base class for test 2 - in - o...,This PE is a base PE for test 2 - in - one - o...,"The TestTwoInOneOut PE, extending GenericPE, t...",0.054102


In [20]:
columnSim('log the metadata from incomming connections', 'full_sum_doc', 1)

,code,code_line_splits,class,summary,mod_summary,full_summary,mod_full_sum,gpt_sum,cosine_similarity_doc
4,\nclass ProvenanceLogger(GenericPE):\n def ...,,ProvenanceLogger,Log the nagios.,Log the nagios.,A logger for the missing sequence sequence.,A logger for the missing sequence sequence.,"The ProvenanceLogger PE, extending the Generic...",0.265153
8,class AFINNSentimeScore(IterativePE):\n def...,,AFinnSentimeScoreCommented2,Process the nltk tag and return the article th...,Process the nltk tag and return the article th...,AFASTIME - SENTIME - SENTIME - SENTIME - S,AFASTIME - SENTIME - SENTIME - SENTIME - S,"The AFINNSentimeScore PE, derived from Iterati...",0.146533
9,class ReadData(GenericPE):\n def __init__(s...,,ReadData,Process the nagios. txt file and return a dict...,Process the nagios. txt file and return a dict...,This class reads the data from the twitter and...,This PE reads the data from the twitter and wr...,"The ReadData PE, extending GenericPE, reads an...",0.129199
5,class TestMultiProducer(GenericPE):\n def _...,,TestMultiProducer,Process the sequence of sequence of sequence o...,Process the sequence of sequence of sequence o...,A class that implements a multi - producer int...,A PE that implements a multi - producer interf...,"The TestMultiProducer PE, extending GenericPE,...",0.112290
7,class AFINNSentimeScore(IterativePE):\n # T...,,AFINNSentimeScoreCommented,This method processes the nltk sentiment file ...,This method processes the nltk sentiment file ...,This class calculates the average sentiment sc...,This PE calculates the average sentiment score...,"The AFINNSentimeScore PE, inherited from Itera...",0.012755


In [21]:
columnSim('test a PE', 'mod_full_sum_doc', 1)

,code,code_line_splits,class,summary,mod_summary,full_summary,mod_full_sum,gpt_sum,cosine_similarity_doc
1,class TestTwoInOneOut(GenericPE):\n \n\n ...,,TestTwoInOneOut,Process the sequence of sequence of sequence o...,Process the sequence of sequence of sequence o...,This class is a base class for test 2 - in - o...,This PE is a base PE for test 2 - in - one - o...,"The TestTwoInOneOut PE, extending GenericPE, t...",0.759402
5,class TestMultiProducer(GenericPE):\n def _...,,TestMultiProducer,Process the sequence of sequence of sequence o...,Process the sequence of sequence of sequence o...,A class that implements a multi - producer int...,A PE that implements a multi - producer interf...,"The TestMultiProducer PE, extending GenericPE,...",0.467826
0,class NumberProducer(GenericPE):\n def __in...,,NumberProducer,Process the sequence of unknown - sequence pro...,Process the sequence of unknown - sequence pro...,A class that produces a sequence of numbers fr...,A PE that produces a sequence of numbers from ...,"The NumberProducer PE, a subclass of GenericPE...",0.442506
9,class ReadData(GenericPE):\n def __init__(s...,,ReadData,Process the nagios. txt file and return a dict...,Process the nagios. txt file and return a dict...,This class reads the data from the twitter and...,This PE reads the data from the twitter and wr...,"The ReadData PE, extending GenericPE, reads an...",0.389175
3,class NumberProducer(ProducerPE):\n def __i...,,RandomNumberProducer,Generate a random seqno from 1 to 1000.,Generate a random seqno from 1 to 1000.,Producer class for producing a random number f...,Producer PE for producing a random number from...,"The NumberProducer PE, a subclass of ProducerP...",0.330582


In [22]:
columnSim('test a PE', 'gpt_sum_doc', 1)

,code,code_line_splits,class,summary,mod_summary,full_summary,mod_full_sum,gpt_sum,cosine_similarity_doc
1,class TestTwoInOneOut(GenericPE):\n \n\n ...,,TestTwoInOneOut,Process the sequence of sequence of sequence o...,Process the sequence of sequence of sequence o...,This class is a base class for test 2 - in - o...,This PE is a base PE for test 2 - in - one - o...,"The TestTwoInOneOut PE, extending GenericPE, t...",0.521036
9,class ReadData(GenericPE):\n def __init__(s...,,ReadData,Process the nagios. txt file and return a dict...,Process the nagios. txt file and return a dict...,This class reads the data from the twitter and...,This PE reads the data from the twitter and wr...,"The ReadData PE, extending GenericPE, reads an...",0.446561
5,class TestMultiProducer(GenericPE):\n def _...,,TestMultiProducer,Process the sequence of sequence of sequence o...,Process the sequence of sequence of sequence o...,A class that implements a multi - producer int...,A PE that implements a multi - producer interf...,"The TestMultiProducer PE, extending GenericPE,...",0.429397
4,\nclass ProvenanceLogger(GenericPE):\n def ...,,ProvenanceLogger,Log the nagios.,Log the nagios.,A logger for the missing sequence sequence.,A logger for the missing sequence sequence.,"The ProvenanceLogger PE, extending the Generic...",0.359839
0,class NumberProducer(GenericPE):\n def __in...,,NumberProducer,Process the sequence of unknown - sequence pro...,Process the sequence of unknown - sequence pro...,A class that produces a sequence of numbers fr...,A PE that produces a sequence of numbers from ...,"The NumberProducer PE, a subclass of GenericPE...",0.318538


In [23]:
# pd.set_option('display.max_colwidth', None)
print(registryLam['mod_full_sum'])

print("\nProcess only summaries\n")
print(registryLam['summary'])

0    A PE that produces a sequence of numbers from ...
1    This PE is a base PE for test 2 - in - one - o...
2    Generate a random word from the sequence of wo...
3    Producer PE for producing a random number from...
4          A logger for the missing sequence sequence.
5    A PE that implements a multi - producer interf...
6             AFASTA AFINNSentimeScore - AFASTA AFINNS
7    This PE calculates the average sentiment score...
8           AFASTIME - SENTIME - SENTIME - SENTIME - S
9    This PE reads the data from the twitter and wr...
Name: mod_full_sum, dtype: object

Process only summaries

0    Process the sequence of unknown - sequence pro...
1    Process the sequence of sequence of sequence o...
2                 Generate a random n - token session.
3              Generate a random seqno from 1 to 1000.
4                                      Log the nagios.
5    Process the sequence of sequence of sequence o...
6    Process the nltk tag and return the average sc...
7    T

ChatGPT outputs using recommended input sentence from https://arxiv.org/pdf/2305.12865.pdf
please generate a short comment in one sentence for the following  PE:

0 The NumberProducer PE, a subclass of GenericPE, generates and outputs a sequence of numbers based on the specified number of iterations.
1 The TestTwoInOneOut PE, extending GenericPE, takes two input connections, combines their values into a single result, and provides it as an output.
2 The RandomWordProducer PE, based on the GenericPE class, generates and provides random words from a predefined list as output.
3 The NumberProducer PE, a subclass of ProducerPE, generates and provides a random number between 1 and 1000 as its output.
4 The ProvenanceLogger PE, extending the GenericPE class, is designed to log metadata and handle exceptions during its processing.
5 The TestMultiProducer PE, extending GenericPE, generates multiple output values based on the specified number of outputs.
6 The AFINNSentimeScore PE, inherited from IterativePE, calculates the sentiment score of text data using the AFINN lexicon and returns the average sentiment score for each article.
7 The AFINNSentimeScore PE, derived from IterativePE, calculates the average sentiment score for text data based on a provided word sentiment file, and it processes articles to determine their sentiment scores.
9 The ReadData PE, extending GenericPE, reads and processes data from a file, extracting text and location information from each line and providing it as output, while keeping track of the number of processed articles.

In [24]:
def line_splits(inputPE):
    lineSplits = []
    lines = inputPE.strip().split("\n")
    for line in lines:
        line = line.strip()
        if line != "":
            lineSplits.append(line)

    return lineSplits
registryLam['code_line_splits'] = registryLam['code'].apply(line_splits)

In [25]:
# function to split up and encode received code line by line
# This makes more sense when considering the use case for code search
# we aren't going to be looking up an entire function
# unless we are looking for plagarism / similarity like repoSim but that's not really the usecase for this is it?
# Especially since the whole function lookup is not brilliant.

def line_encode(inputPE, model_type):
    print(inputPE)
    lineEmbeddings = []
    
    for line in inputPE:
       
        lineEmbeddings.append(encode(line, model_type))
        
    return lineEmbeddings


registryLam['code_line_embeddings'] = registryLam['code_line_splits'].apply(line_encode, model_type=1)


['class NumberProducer(GenericPE):', 'def __init__(self, numIterations=1):', 'GenericPE.__init__(self)', 'self.outputconnections = {"output": {NAME: "output", TYPE: ["number"]}}', 'self.counter = 0', 'self.numIterations = numIterations', 'def process(self, inputs):', 'self.counter += 1', 'for i in range(self.numIterations):', 'self.write("output", f"{self.counter}:{i}")']
['class TestTwoInOneOut(GenericPE):', 'def __init__(self):', 'GenericPE.__init__(self)', 'self.inputconnections = {"input0": {NAME: "input0"}, "input1": {NAME: "input1"}}', 'self.outputconnections = {"output": {NAME: "output", TYPE: ["result"]}}', 'def process(self, inputs):', 'result = ""', 'for inp in self.inputconnections:', 'if inp in inputs:', 'result += str(inputs[inp])', 'if result:', 'return {"output": result}']
['class RandomWordProducer(GenericPE):', 'words = [', '"Dispel4Py",', '"Earthquake",', '"Computing",', '"Seismology",', '"Modelling",', '"Analysis",', '"Infrastructure",', ']', 'def __init__(self):', '

In [26]:
# code search on lines
def get_matching_line(inputList, lineNum):
    return inputList[lineNum]


def lineSim(inputString, modelType):

    #NEW -- indicate the model_type=1 for unixcoder (text-to-code)
    user_query_docs_emb = encode(inputString, model_type=modelType)

    # Compute cosine similarity
    user_query_emb = np.array(user_query_docs_emb)
    best_overall = []
    best_pos = []
    matching_lines = []
    for pe in registryLam['code_line_embeddings']:
        result = cosine_similarity(user_query_emb.reshape(1, -1), np.vstack(pe))[0]
        best_within_pe = max(result)
        
        best_pos.append((list(result)).index(best_within_pe))
        
        best_overall.append(best_within_pe)

    
    
    cos_similarities = cosine_similarity(user_query_emb.reshape(1, -1), np.vstack(registryLam['sum_doc']))
    
    # Add cosine similarity scores as a new column
    registry_doc=registryLam.copy().drop(['code_embedding', 'code_line_embeddings','process', 'sum_doc', 'mod_sum_doc','full_sum_doc', 'mod_full_sum_doc', 'gpt_sum_doc'], axis=1)
    # registry_doc.drop('code', axis=1)
    registry_doc["cosine_similarity_doc"] = best_overall
    registry_doc['best_pos'] = best_pos
    registry_doc["best_result_line"] = registry_doc.apply(lambda x: get_matching_line(x.code_line_splits, x.best_pos), axis=1)
    registry_doc = registry_doc.drop(['code_line_splits', 'best_pos'], axis=1)
    # Sort the dataframe based on cosine similarity
    sorted_df = registry_doc.sort_values(by="cosine_similarity_doc", ascending=False)

    # Retrieve the top 5 most similar documents
    top_5_similar_docs = sorted_df.head(5)
    return top_5_similar_docs

In [35]:
# Expecting ReadData
columnSim(''' for line in article_file:
            fields = line.split("\t")
            location = fields[3]
            text = fields[4]
            self.count += 1
            return_article = {"text": text, "location": location}
            # print("TEXT %s", text)
            self.write("output", return_article)''', 'code_embedding', 2)['class']

1         TestTwoInOneOut
5       TestMultiProducer
3    RandomNumberProducer
9                ReadData
2      RandomWordProducer
Name: class, dtype: object

In [34]:
# Expecting ReadData
lineSim(''' for line in article_file:
            fields = line.split("\t")
            location = fields[3]
            text = fields[4]
            self.count += 1
            return_article = {"text": text, "location": location}
            # print("TEXT %s", text)
            self.write("output", return_article)''', 1)['class']


9                       ReadData
6              AFINNSentimeScore
7     AFINNSentimeScoreCommented
8    AFinnSentimeScoreCommented2
5              TestMultiProducer
Name: class, dtype: object

In [33]:
# Expecting AFINNSentimeScore
lineSim('''if (is_encoded):''', 1)['class']

6              AFINNSentimeScore
7     AFINNSentimeScoreCommented
8    AFinnSentimeScoreCommented2
1                TestTwoInOneOut
4               ProvenanceLogger
Name: class, dtype: object

In [32]:
# Expecting AFINNSentimeScore
display(columnSim('if (is_encoded):', 'code_embedding', 2)['class'])

1         TestTwoInOneOut
4        ProvenanceLogger
3    RandomNumberProducer
5       TestMultiProducer
9                ReadData
Name: class, dtype: object